In [3]:
import torch
import sys
import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from sklearn import model_selection
from sklearn import metrics
from sklearn.metrics import roc_auc_score, log_loss, roc_auc_score, roc_curve, auc
from sklearn.model_selection import StratifiedKFold, ShuffleSplit, cross_val_score, train_test_split

print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
from subprocess import call
# call(["nvcc", "--version"]) does not work
! nvcc --version
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
# call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

print ('Available devices ', torch.cuda.device_count())
print ('Current cuda device ', torch.cuda.current_device())

import numpy
import numpy as np

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

import pandas
import pandas as pd

import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)
%matplotlib inline

# !pip install psutil
import psutil
import os
def cpuStats():
        print(sys.version)
        print(psutil.cpu_percent())
        print(psutil.virtual_memory())  # physical memory usage
        pid = os.getpid()
        py = psutil.Process(pid)
        memoryUse = py.memory_info()[0] / 2. ** 30  # memory use in GB...I think
        print('memory GB:', memoryUse)

cpuStats()

__Python VERSION: 3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]
__pyTorch VERSION: 1.4.0
__CUDA VERSION
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2017 NVIDIA Corporation
Built on Fri_Nov__3_21:07:56_CDT_2017
Cuda compilation tools, release 9.1, V9.1.85
__CUDNN VERSION: 7603
__Number CUDA Devices: 2
__Devices
Active CUDA Device: GPU 0
Available devices  2
Current cuda device  0
3.7.4 (default, Aug 13 2019, 20:35:49) 
[GCC 7.3.0]
24.0
svmem(total=67477729280, available=32392196096, percent=52.0, used=31103012864, free=8581877760, active=32186486784, inactive=19827380224, buffers=267857920, cached=27524980736, shared=3240423424, slab=5197410304)
memory GB: 0.2600975036621094


In [2]:
use_cuda = torch.cuda.is_available()
# use_cuda = False

FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

/home/raznem/projects_artur


In [5]:
cd ../projects/audio-pipelines

/home/raznem/projects/audio-pipelines


In [6]:
os.chdir('data')

In [7]:
!cd VoiceChallengeTestSet-8k/data_eval/sejm_voicelab_eval;ls

eval_set  test


In [6]:
# !cd VoiceChallengeTestSet-8k/data_eval/sejm_voicelab_eval/eval_set;wget http://naan.mimuw.edu.pl/~pzuk/targets;ls

--2020-01-19 00:52:42--  http://naan.mimuw.edu.pl/~pzuk/targets
Resolving naan.mimuw.edu.pl (naan.mimuw.edu.pl)... 193.0.109.30
Connecting to naan.mimuw.edu.pl (naan.mimuw.edu.pl)|193.0.109.30|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57998 (57K)
Saving to: ‘targets.1’

targets.1           100%[===================>]  56,64K  --.-KB/s    in 0,005s  

2020-01-19 00:52:42 (11,3 MB/s) - ‘targets.1’ saved [57998/57998]

hyps  score.py	targets  targets.1  text  utt2spk  wav.scp


In [7]:
!cd VoiceChallengeTestSet-8k/data_eval/sejm_voicelab_eval/eval_set;rm targets.1;ls

hyps  score.py	targets  text  utt2spk	wav.scp


In [6]:
base = 'VoiceChallengeTestSet-8k/data_eval/sejm_voicelab_eval/eval_set/'

hyps = pd.read_csv(base+"hyps", names=['audio_id'])
hyps[['audio_id','hyps']] = hyps["audio_id"].str.split(" ", 1, expand=True)

targets = pd.read_csv(base+"targets", names=['audio_id'])
targets[['audio_id','target']] = targets["audio_id"].str.split(" ", 1, expand=True)

text = pd.read_csv(base+"text", names=['audio_id'])
text[['audio_id','text']] = text["audio_id"].str.split(" ", 1, expand=True)

speakers = pd.read_csv(base+"utt2spk", names=['audio_id'])
speakers[['audio_id','speaker']] = speakers["audio_id"].str.split(" ", 1, expand=True)

images = pd.read_csv(base+"wav.scp", names=['audio_id'])
images[['audio_id','audio_path']] = images["audio_id"].str.split(" ", 1, expand=True)

dataset = pd.merge(hyps, images, how="left")

In [7]:
def remove_absolute(string, prefix='/home/raznem/projects/audio-pipelines/data/'):
    if string.startswith(prefix):
        string = string[len(prefix):]
    return string

dataset = pd.merge(hyps, text, how="left")
dataset = pd.merge(dataset, speakers, how="left")
dataset = pd.merge(dataset, images, how="left")
dataset = pd.merge(dataset, targets, how="left")

In [9]:
dataset.head()

,audio_id,hyps,text,speaker,audio_path,target,image_path
0,Adam_Andruszkiewicz_4_25_33-3,stronę dzień referendalny ustawy pel wydrukuje...,stronę dzień referendalny kropka p 50 wydrukuj...,Adam_Andruszkiewicz,data_eval/sejm_voicelab_eval/test/Adam_Andrusz...,1,data_eval/sejm_voicelab_eval/test/Adam_Andrusz...
1,Adam_Cyranski_2_2_56-6,czy nato propozycje klubu nowoczesna idą w dwó...,szczyt nato propozycje klubu nowoczesna idą w ...,Adam_Cyranski,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...,0,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...
2,Adam_Cyranski_4_26_31-8,mechanizmy zostaną wprowadzone aby żołnierze m...,mechanizmy zostaną wprowadzone aby żołnierze m...,Adam_Cyranski,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...,1,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...
3,Adam_Cyranski_4_42_16-1,szeregowych dzisiaj wprowadzamy ustawę która w...,szeregowych dzisiaj wprowadzamy ustawę która w...,Adam_Cyranski,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...,1,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...
4,Adam_Cyranski_4_50_4-0,panie pośle proszę zadawać pytanie potem pan b...,panie pośle proszę zadawać pytanie potem pan b...,Adam_Cyranski,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...,0,data_eval/sejm_voicelab_eval/test/Adam_Cyransk...


In [10]:
from PIL import Image
im = Image.open('data_eval/sejm_voicelab_eval/test/Adam_Cyranski_4_26_31-8.png')
im.size

(400, 400)

In [14]:
# dataset.to_csv('data/dataset_val.csv')

In [57]:
dataset['image_path'][0]

'data_eval/sejm_voicelab_eval/test/Adam_Andruszkiewicz_4_25_33-3.png'

In [10]:
from allennlp.commands.elmo import ElmoEmbedder
from torchvision import transforms

try:
    from PIL import Image
except ImportError:
    import Image
    
toImg = transforms.ToPILImage()
toTensor = transforms.ToTensor()
    
class WavImagesLoader(Dataset):
    def __init__(self, csv_path, transform=None):
        self.dataset = pd.read_csv(csv_path)
        self.transform = transform
        
    def __getitem__(self, key):
        image_path = self.dataset['image_path'][key]
        image = Image.open(image_path)
        image = image.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
        
        return image
    
    def __len__(self):
        return len(self.dataset)
    
    @staticmethod        
    def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))    

#     @staticmethod    
#     def flaotTensorToImage(img, mean=0, std=1):
#         """convert a tensor to an image"""
#         img = np.transpose(img.numpy(), (1, 2, 0))
#         img = (img*std+ mean)*255
#         img = img.astype(np.uint8)    
#         return img    
    
    
class ElmoWavImagesLoader(WavImagesLoader):
    def __init__(self, csv_path, elmo_model, transform=None):
        super().__init__(csv_path, transform=None)
        self.elmo_model = elmo_model
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_path = self.dataset['image_path'][key]
        image = Image.open(path)
        image = img.convert('RGB')
        image = toTensor(image)
        if self.transform is not None:
            image = self.transform(image)
            
        text = self.dataset['hyps'][key]
        text = text.split(' ')
        embedding = self.elmo_model(text)
        embedding = embedding.sum(axis=0)

        return image, embedding, target
    
    
class ElmoWavVecLoader(ElmoWavImagesLoader):
    def __init__(self, csv_path, elmo_model, transform=None):
        super().__init__(csv_path, elmo_model, transform=None)
        
    def __getitem__(self, key):
        target = self.dataset['target'][key]
        image_vec = self.dataset['image_vec'][key]
        text = self.dataset['hyps'][key]
        text = text.split(' ')
        embedding = self.elmo_model(text)
        embedding = embedding.sum(axis=0)

        return image_vec, embedding, target

INFO:pytorch_pretrained_bert.modeling:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.modeling_bert:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:pytorch_transformers.modeling_xlnet:Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
INFO:allennlp.common.registrable:instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>


## Conv net to get vector from Image

In [11]:
dropout = torch.nn.Dropout(p=0.30)
class ConvRes(nn.Module):
    def __init__(self, insize, outsize):
        super(ConvRes, self).__init__()
        drate = .3
        self.math = nn.Sequential(
            nn.BatchNorm2d(insize),
            # nn.Dropout(drate),
            torch.nn.Conv2d(insize, outsize, kernel_size=2, padding=2),
            nn.PReLU(),
        )

    def forward(self, x):
        return self.math(x)


class ConvCNN(nn.Module):
    def __init__(self, insize, outsize, kernel_size=7, padding=2, pool=2, avg=True):
        super(ConvCNN, self).__init__()
        self.avg = avg
        self.math = torch.nn.Sequential(
            torch.nn.Conv2d(insize, outsize, kernel_size=kernel_size, padding=padding),
            torch.nn.BatchNorm2d(outsize),
            torch.nn.LeakyReLU(),
            torch.nn.MaxPool2d(pool, pool),
        )
        self.avgpool = torch.nn.AvgPool2d(pool, pool)

    def forward(self, x):
        x = self.math(x)
        if self.avg is True:
            x = self.avgpool(x)
        return x


class Net(nn.Module):
    def __init__(self, output_dim):
        super(Net, self).__init__()

        self.avgpool = nn.AdaptiveAvgPool2d(1)

        self.cnn1 = ConvCNN(3, 32, kernel_size=7, pool=4, avg=False)
        self.cnn2 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)
        self.cnn3 = ConvCNN(32, 32, kernel_size=5, pool=2, avg=True)

        self.res1 = ConvRes(32, 64)

        self.features = nn.Sequential(
            self.cnn1, dropout,
            self.cnn2,
            self.cnn3,
            self.res1,
        )

        self.classifier = torch.nn.Sequential(
            nn.Linear(5184, output_dim),
        )
        self.sig = nn.Sigmoid()
  
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        x = self.sig(x)
        return x

    
# if use_cuda:
#     lgr.info ("Using the GPU")
#     model = Net(output_dim).cuda() # On GPU
# else:
#     lgr.info ("Using the CPU")
#     model = Net(output_dim) # On CPU

# lgr.info('Model {}'.format(model))

In [12]:
import gc
def image_trainer(loader_train, model, optimizer, save_every=None, print_every=None, epochs=1, use_gpu=True, 
                 dtype=torch.float32):
    if use_gpu and torch.cuda.is_available():
        device = torch.device('cuda:1')
    else:
        device = torch.device('cpu')
        
    model.train()
    model.to(device=device)

#     for e in range(epochs):
#         print('Epoch %d' %e)
#         for t, (x, y) in enumerate(loader_train):
#             x = x.to(device=device, dtype=dtype)
#             y = y.to(device=device, dtype=dtype)

#             y_pred = model(x)
#             y_pred = torch.squeeze(y_pred)
#             loss = loss_fn(y_pred, y)

#             optimizer.zero_grad()
#             loss.backward()

#             optimizer.step()

#             if print_every is not None and t % print_every == 0:
#                 print('Iteration %d, loss = %.4f; epoch %d' % (t, loss.item(), e))
                
#         if save_every is not None and t % save_every == 0:
#             torch.save(model.state_dict(), 'models/e%d_dupa_cnn.pt' % e)
#             gc.collect()
    torch.save(model.state_dict(), '/tmp/inference_draft.pt')

In [13]:
loss_fn = torch.nn.BCELoss()

In [18]:
torch.cuda.empty_cache()

In [14]:
BATCH_SIZE = 128
LEARNING_RATE = 1e-4
USE_GPU = True
TE_DTYPE = torch.float32
TARGET_DIM = 1
NUM_EPOCHS = 100

csv_path = 'data/dataset_val.csv'
model = Net(TARGET_DIM)
image_data = WavImagesLoader(csv_path)
dataloader = DataLoader(image_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)


In [15]:
device = "cuda:0"
model.load_state_dict(torch.load('models/image_cnn_100e_e70_image_cnn.pt'))
model = model.eval().to(device)

In [16]:
prediction_list = []
with torch.no_grad():
    for i, batch in enumerate(dataloader):
        pred = model(batch.to(device)).to("cpu")
        prediction_list.append(pred)

In [51]:
result = torch.squeeze(torch.cat(prediction_list))

In [45]:
# result = (result > 0.9).type(torch.int)

In [46]:
df = pd.read_csv(csv_path)
df.head()
df["result"] = result
result = df[["audio_id", "hyps", "text", "target", "result"]].copy()
# # result.to_csv("submission_2239.txt", header=False, sep=" ", index=False)

In [49]:
import numpy as np
def rval_acc(df):
    df['acc'] = df['target'] == df['result']
    acc = df['acc'].sum()/len(df['acc'])
    return acc

def find_thresh(df):
    max_acc = 0.0
    best = 0
    for tr in np.arange(0.1,1,0.1):
        df_0 = (df > tr).type(torch.int)
        df_1 = pd.read_csv(csv_path)
        df_1["result"] = df_0
        df_2 = df_1[["audio_id", "hyps", "text", "target", "result"]].copy()
        acc = rval_acc(df_2)
        if acc>max_acc:
            max_acc = acc
            best = tr
        print(f'Threshold: {tr}, acc: {acc}')
    return(max_acc, best)

In [28]:
sum(result)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [41]:
prediction_list[:10]

[tensor([[1.1295e-01],
         [3.1939e-01],
         [4.7646e-03],
         [8.2300e-01],
         [3.6138e-01],
         [6.7834e-01],
         [8.9241e-01],
         [9.9536e-01],
         [6.7618e-01],
         [9.9773e-01],
         [1.6123e-03],
         [6.1922e-02],
         [7.0179e-01],
         [9.9996e-01],
         [8.5671e-01],
         [5.4975e-02],
         [9.9960e-01],
         [9.9009e-01],
         [5.2179e-02],
         [9.0659e-01],
         [4.8412e-01],
         [9.9521e-01],
         [4.7270e-01],
         [1.7834e-02],
         [9.8809e-01],
         [2.9266e-06],
         [3.4489e-02],
         [9.5738e-01],
         [2.0221e-03],
         [5.0348e-01],
         [9.8034e-01],
         [9.9976e-01],
         [9.7026e-01],
         [8.4788e-01],
         [2.2412e-01],
         [1.1598e-01],
         [9.7839e-01],
         [1.3701e-01],
         [9.9257e-01],
         [9.9957e-01],
         [9.9253e-01],
         [5.1313e-02],
         [2.6000e-02],
         [8